<a target="_blank" href="https://colab.research.google.com/github/sonder-art/mhar/blob/released/nbs/tutorial_nonfull_dimensional.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This tutorial will show how to use `mhar` for sampling polytopes. It is focused on executing parallel MCMC walks over a polytope in GPUs.

In [1]:
import torch

Before starting let's check if you have an avaialble gpu device or not.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [3]:
# We will choose 64-bits
dtype = torch.float64

## Canonical Representation

The polytope in question must be presented in matrix canonical representation (as opposed to vertex). `mhar` assumes that the matrix has no repeated or redundant restrictions.

## Non-Fully dimensional Polytopes  

### Definition

> $A^IX \leq b^I$  
> 
> $A^EX = b^E$


For non-fully dimensional polytopes we need to use the class `NFDPolytope` in the `mhar.polytope` module. The restrictions must be passed as pytorch tensors.  
  
We will sample the unit hypercube that is defined as:  
> $n-simplex = \{x \in R^n || \sum_{i=1}^{n} x_i = 1, 0 \leq x_i \} $  

Which we can represent in matrix restrictions:  
$ -Ix \leq 0$  
$ [1]^n = 1 $  
Where $I$ is the identity matrix of dimension $n \times n$ 


#### Definition-Code

Lets create the tensors to represent the restrictions that define the polytope. Since we need to create a projection matrix for the non-fully dimensional object we need to preserve the numerical stability of the algorithm, we suggest using 64 bits precision. 

In [4]:
n = 3 # Dimension
dtype = torch.float64 # Precision 
A_I = torch.eye(n).to(dtype) * -1.0
b_I = torch.empty(n, 1, dtype=dtype)
b_I.fill_(0.0)

# Create Equalities
A_E = torch.empty(1, n, dtype=dtype)
A_E.fill_(1.0)
b_E = torch.empty(1, 1, dtype=dtype)
b_E.fill_(1.0)      
print(f'Inequality Matrix A^I \n {A_I} \n')
print(f'Inequality Vector b^I \n {b_I}\n')
print(f'Equality Matrix A^E \n {A_E} \n')
print(f'Equality Vector b^E \n {b_E}')

Inequality Matrix A^I 
 tensor([[-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]], dtype=torch.float64) 

Inequality Vector b^I 
 tensor([[0.],
        [0.],
        [0.]], dtype=torch.float64)

Equality Matrix A^E 
 tensor([[1., 1., 1.]], dtype=torch.float64) 

Equality Vector b^E 
 tensor([[1.]], dtype=torch.float64)


Now lets create a `NFDPolytope` object to represent the polytope.

In [5]:
from mhar.polytope import NFDPolytope
simplex = NFDPolytope(A_I, # Inequality Restriction Matrix 
                     b_I,  # Inequality Vector
                     A_E, # Equality Restriction Matrix 
                     b_E,  # Equality Vector
                     dtype, # torch dtype
                     device, # device used cpu or cuda
                     copy=False # bool for creating a copy of the restrictions
                     )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [6]:
simplex

Numeric Precision (dtype) torch.float64
Device: cuda
A_in: torch.Size([3, 3]) 
b_in: torch.Size([3, 1])
A_eq: torch.Size([1, 3]) 
b_eq: torch.Size([1, 1])

### Projection Matrix

Now we need to compute que projection matrix that we will use for projecting the random directions vectors to the equality space. For that we can use the method `NFDPolytope.compute_projection_matrix()`. We recommend using the highest precision possible to compute this matrix.

In [7]:
simplex.compute_projection_matrix(device=device, solver_precision=torch.float64)

Max non zero error for term (A A')^(-1)A at precision torch.float64:  tensor(0., device='cuda:0', dtype=torch.float64)


In [8]:
simplex

Numeric Precision (dtype) torch.float64
Device: cuda
A_in: torch.Size([3, 3]) 
b_in: torch.Size([3, 1])
A_eq: torch.Size([1, 3]) 
b_eq: torch.Size([1, 1])
Projection Matrix: torch.Size([3, 3])

### Starting Inner Point(s)

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [9]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In [10]:
x0 = ChebyshevCenter(polytope=simplex, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device='cuda', # device used cpu or cuda
                    solver_precision=np.float64 # numpy dtype
                    )


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


In [11]:
x0

tensor([[0.3333],
        [0.3333],
        [0.3333]], device='cuda:0', dtype=torch.float64)

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [12]:
x0 = torch.cat([x0, 
            torch.tensor([[.25], [.25], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[0.3333, 0.2500],
        [0.3333, 0.2500],
        [0.3333, 0.5000]], device='cuda:0', dtype=torch.float64)

Now we can proceed to sample the `polytope`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type `Polytope` or `NFDPolytope` that defines it.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [13]:
x0

tensor([[0.3333, 0.2500],
        [0.3333, 0.2500],
        [0.3333, 0.5000]], device='cuda:0', dtype=torch.float64)

In [14]:
from mhar.walk import walk
X = walk(polytope=simplex,
        X0 = x0,  
        z=100, 
        T=10, 
        warm=0,
        thinning=n**3, 
        device=device, 
        seed=None,
        verbosity=2
)

Minimum number allowed -1.7976931348623157e+308
Maximum number allowed 1.7976931348623157e+308
Eps:  2.220446049250313e-16
Values close to zero will be converted to 3eps or -3eps: 6.661338147750939e-16
n:  3   mI: 3   mE: 1   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |███---------------------------| 10.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████------------------------| 20.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |█████████---------------------| 30.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |████████████------------------| 40.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |███████████████---------------| 50.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████------------| 60.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid sa

`walk` produces `T` $\times$ `z` uncorrelated points. It returns a vector of dimension `T` $\times$ `z` $\times$ `n`.  

In [27]:
X

tensor([[[1.8518e-01, 2.1163e-01, 4.6898e-01,  ..., 1.9357e-01,
          2.4413e-02, 6.3012e-02],
         [2.7464e-01, 2.8838e-01, 1.3008e-01,  ..., 4.1514e-01,
          7.2643e-01, 7.8429e-01],
         [5.4018e-01, 4.9998e-01, 4.0094e-01,  ..., 3.9128e-01,
          2.4915e-01, 1.5270e-01]],

        [[1.2658e-01, 3.2733e-01, 4.3178e-01,  ..., 1.2386e-01,
          2.4358e-01, 3.4306e-01],
         [4.7800e-01, 1.7548e-01, 1.9821e-01,  ..., 4.4128e-02,
          5.1164e-01, 5.1802e-02],
         [3.9542e-01, 4.9719e-01, 3.7001e-01,  ..., 8.3201e-01,
          2.4479e-01, 6.0514e-01]],

        [[4.0546e-01, 4.9163e-01, 1.1899e-02,  ..., 3.3081e-01,
          4.2431e-01, 3.9564e-01],
         [5.4480e-01, 6.7977e-02, 3.8142e-01,  ..., 3.9524e-01,
          3.9749e-01, 3.3416e-01],
         [4.9740e-02, 4.4039e-01, 6.0668e-01,  ..., 2.7395e-01,
          1.7820e-01, 2.7020e-01]],

        ...,

        [[7.0043e-01, 2.9754e-01, 4.6188e-01,  ..., 5.4753e-01,
          1.5119e-01, 6.2

In [28]:
X.shape

torch.Size([10, 3, 100])

In [31]:
X.sum(1)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1

In [32]:
X.shape

torch.Size([10, 3, 100])

### Numerical Stability

There is some numerical inestability that causes the algorithm to degrade overtime, we recommend checking your walk everyonce in a while and be sure that it is not a big deal. The inestability is due to the projection matrix.

In [36]:
tol = 1e-10
print('Infinite: ', (~torch.isfinite(X)).sum())
print('Nans:  ',(torch.isnan(X)).sum())
print('Inequality violation:  ',(X.sum(1)!=1.0).sum())
print(f'Inequality violation with tol {tol}:  ',((X.sum(1)- 1.0).abs()>tol).sum())

Infinite:  tensor(0)
Nans:   tensor(0)
Inequality violation:   tensor(938)
Inequality violation with tol 1e-10:   tensor(0)


### Summary

To sumamrize the steps taken we can use the `polytope_examples` for creating a `Hypercube`.

In [19]:
from mhar.polytope_examples import Simplex

# Create a polytope (Simplex)
simplex_sim = Simplex(10,
                      dtype=torch.float64,
                      device='cuda'
                      )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [20]:
simplex_sim.compute_projection_matrix(device='cuda')


Max non zero error for term (A A')^(-1)A at precision torch.float64:  tensor(2.2204e-16, device='cuda:0', dtype=torch.float64)


Define/Find inner points

In [21]:
x0_sim = ChebyshevCenter(polytope=simplex_sim, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device='cuda',
                    solver_precision=np.float64)




Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


Sample points

In [22]:
X_sim = walk(polytope=simplex_sim,
        X0 = x0_sim,  
        z=100, 
        T=1, 
        warm=0,
        thinning=10000, 
        device= None, 
        seed=None,
        verbosity=2
)
X_sim

Minimum number allowed -1.7976931348623157e+308
Maximum number allowed 1.7976931348623157e+308
Eps:  2.220446049250313e-16
Values close to zero will be converted to 3eps or -3eps: 6.661338147750939e-16
n:  10   mI: 10   mE: 1   z: 100


% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


tensor([[[2.2232e-01, 4.2157e-02, 1.0623e-01, 9.0542e-02, 1.1183e-01,
          1.1443e-01, 9.0691e-02, 1.2217e-01, 1.0717e-01, 2.0869e-01,
          6.9864e-02, 2.6897e-02, 2.3011e-01, 8.0364e-02, 2.4504e-01,
          3.0917e-01, 1.3760e-01, 4.4741e-02, 7.0836e-02, 8.0677e-03,
          7.6785e-02, 2.9897e-01, 1.6811e-01, 6.2855e-02, 1.7506e-02,
          1.3156e-02, 3.1130e-02, 4.4690e-02, 1.4000e-01, 2.3178e-01,
          1.3229e-01, 2.9500e-02, 2.0480e-01, 8.4248e-02, 1.2623e-01,
          1.2501e-03, 2.4541e-02, 1.4675e-01, 2.5414e-02, 1.2247e-01,
          7.6131e-02, 2.0446e-01, 4.8699e-02, 7.3948e-02, 2.0779e-01,
          1.6323e-01, 8.6012e-03, 1.1459e-01, 4.7574e-02, 6.6901e-03,
          8.9183e-02, 5.7458e-02, 1.4460e-01, 2.7484e-01, 9.0192e-02,
          6.5844e-02, 4.1337e-02, 3.1645e-02, 2.2144e-01, 1.2286e-01,
          6.4789e-02, 9.2737e-02, 1.3122e-01, 1.3373e-01, 5.8444e-02,
          2.1104e-02, 7.4213e-02, 6.5096e-03, 7.9778e-02, 8.6577e-02,
          9.3164e-03

In [23]:
X_sim.shape

torch.Size([1, 10, 100])

In [37]:
tol = 1e-10
print('Infinite: ', (~torch.isfinite(X)).sum())
print('Nans:  ',(torch.isnan(X)).sum())
print('Inequality violation:  ',(X.sum(1)!=1.0).sum())
print(f'Inequality violation with tol {tol}:  ',((X.sum(1)- 1.0).abs()>tol).sum())

Infinite:  tensor(0)
Nans:   tensor(0)
Inequality violation:   tensor(938)
Inequality violation with tol 1e-10:   tensor(0)
